In [1]:
import pandas as pd
import numpy as np
from sklearn import *
import matplotlib.pyplot as plt
import xgboost as xgb
from pandas.plotting import scatter_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_classif


In [2]:
# Unpacking data from pickle file and basic information about the data
data = pd.read_pickle('./ass2.pickle')
train, dev, test = data['train'], data['dev'], data['test'] 
print(f"number of features: {len(train.columns) - 1}")
print(f"types of labels: {train['target'].unique()}")

print(f"number of rows in train: {len(train)}")
print(f"number of rows in dev: {len(dev)}")
print(f"number of rows in test: {len(test)}")


number of features: 42
types of labels: [2 1 0]
number of rows in train: 40533
number of rows in dev: 13512
number of rows in test: 13512


In [3]:
#Checking for missing values
train.isnull().sum()

f0        0
f1        0
f2        0
f3        0
f4        0
f5        0
f6        0
f7        0
f8        0
f9        0
f10       0
f11       0
f12       0
f13       0
f14       0
f15       0
f16       0
f17       0
f18       0
f19       0
f20       0
f21       0
f22       0
f23       0
f24       0
f25       0
f26       0
f27       0
f28       0
f29       0
f30       0
f31       0
f32       0
f33       0
f34       0
f35       0
f36       0
f37       0
f38       0
f39       0
f40       0
f41       0
target    0
dtype: int64

In [4]:
# Information about the training data
train.groupby('target').describe()

f0                                                    f1  \
          count      mean       std  min  25%  50%  75%  max    count   
target                                                                  
0        3917.0  1.061527  0.833138  0.0  0.0  1.0  2.0  2.0   3917.0   
1        9882.0  1.082676  0.908831  0.0  0.0  1.0  2.0  2.0   9882.0   
2       26734.0  0.899080  0.798188  0.0  0.0  1.0  2.0  2.0  26734.0   

                  ...  f40           f41                                     \
            mean  ...  75%  max    count      mean       std  min  25%  50%   
target            ...                                                         
0       0.669135  ...  0.0  2.0   3917.0  0.003574  0.074868  0.0  0.0  0.0   
1       0.572759  ...  0.0  2.0   9882.0  0.000911  0.038952  0.0  0.0  0.0   
2       0.509052  ...  0.0  2.0  26734.0  0.001272  0.037681  0.0  0.0  0.0   

                  
        75%  max  
target            
0       0.0  2.0  
1       0.0  2.0  
2       0.0  2.0  

[3 rows x 336 columns]

In [5]:
# Splitting the data into features and labels dataframes
train_features, train_labels = train.drop('target', axis=1), train['target']
dev_features, dev_labels = dev.drop('target', axis=1), dev['target']
test_features, test_labels = test.drop('target', axis=1), test['target']

In [6]:
def model_grid_search(model, param_grid):
    """
    This functions performs grid search on model hyperparameters    
    :param model: A model object
    :param param_grid: dictionary of {hyperparam_1 : [value_1, ..., value_n], ..., hyperparam_n : [value_1, ..., value_n]}
    :return: The model with the hyperparameters that gives the best accuracy.
    """
    k_features = [30, 35, 40, 42]
    GS = GridSearchCV(estimator=model, param_grid=param_grid, verbose=5)
    best_model = None
    best_accuracy = float('-inf')
    best_features = None
    for k in k_features:
        s = SelectKBest(score_func=mutual_info_classif, k=k)
        new_train = s.fit_transform(train_features, train_labels)
        cur_features = s.get_feature_names_out()
        print(f"Performing Grid Search for {k} features: {cur_features}")
        cur_model = GS.fit(new_train, train_labels)
        cur_acc = GS.best_score_
        if best_accuracy < cur_acc:
            best_accuracy = cur_acc
            best_model = cur_model
            best_features = cur_features
    return best_model, best_features

In [8]:
# Check XGBoost
xg_params = {"learning_rate": [0.4, 0.3, 0.2], "gamma": [0, 1], "lambda": [1, 2, 3], "n_estimators": [100, 125, 150, 175]}
xg_model = xgb.XGBClassifier(objective='multi:softmax', num_class=3)
best_xg_model, best_xg_features = model_grid_search(xg_model, xg_params)
y_pred = best_xg_model.predict(dev[best_xg_features])
print("Accuracy:", metrics.accuracy_score(dev_labels, y_pred))
print("Best model hyperparameters:", best_xg_model.best_estimator_)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

Performing Grid Search for 30 features: ['f0' 'f1' 'f6' 'f7' 'f8' 'f10' 'f11' 'f12' 'f13' 'f14' 'f16' 'f18' 'f19'
 'f20' 'f21' 'f23' 'f24' 'f25' 'f26' 'f27' 'f28' 'f30' 'f31' 'f32' 'f33'
 'f34' 'f35' 'f36' 'f37' 'f41']
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END gamma=0, lambda=1, learning_rate=0.4, n_estimators=100;, score=0.816 total time=   0.6s
[CV 2/5] END gamma=0, lambda=1, learning_rate=0.4, n_estimators=100;, score=0.820 total time=   0.5s
[CV 3/5] END gamma=0, lambda=1, learning_rate=0.4, n_estimators=100;, score=0.813 total time=   0.6s
[CV 4/5] END gamma=0, lambda=1, learning_rate=0.4, n_estimators=100;, score=0.819 total time=   0.6s
[CV 5/5] END gamma=0, lambda=1, learning_rate=0.4, n_estimators=100;, score=0.819 total time=   0.6s
Performing Grid Search for 35 features: ['f0' 'f1' 'f2' 'f3' 'f5' 'f6' 'f7' 'f8' 'f10' 'f12' 'f13' 'f14' 'f15'
 'f16' 'f18' 'f19' 'f20' 'f22' 'f24' 'f25' 'f26' 'f27' 'f29' 'f30' 'f31'
 'f32' 'f33' 'f34' 'f35' 'f36' 'f

array([[ 255,  376,  639],
       [ 111, 2661,  601],
       [ 103,  357, 8409]], dtype=int64)

In [ ]:
# Check Random Forest
tree_params = {"max_depth": [None, 30, 50], "n_estimators": [100, 125, 150, 175], "min_samples_split":[2, 3, 4]}
tree_model = RandomForestClassifier()
best_tree_model, best_tree_features = model_grid_search(tree_model, tree_params)
y_pred = best_tree_model.predict(dev[best_tree_features])
print("Accuracy:", metrics.accuracy_score(dev_labels, y_pred))
print("Best model hyperparameters:", best_tree_model.best_estimator_)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

In [ ]:
# Check Gradient Boost
gb_params = {"max_depth": [2, 3], "n_estimators": [100, 125, 150, 175], "min_samples_split": [2, 3], "learning_rate": [0.1, 0.01]}
gb_model = GradientBoostingClassifier()
best_gb_model, best_gb_features = model_grid_search(gb_model, gb_params)
y_pred = best_gb_model.predict(dev[best_gb_features])
print("Accuracy:", metrics.accuracy_score(dev_labels, y_pred))
print("Best model hyperparameters:", best_gb_model.best_estimator_)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(k=30)),
                 ('classifier', xgb.XGBClassifier(objective='multi:softmax', num_class=3))])

search_space = [{'selector__k': [30, 35, 40, 42]},
                {'classifier': [RandomForestClassifier()],
                 'classifier__n_estimators': [100, 125, 150, 175],
                 'classifier__max_depth': [None, 30, 50],
                 'classifier__min_samples_split': [2, 3, 4]},
                {'classifier': [GradientBoostingClassifier()],
                 'classifier__learning_rate': [0.1, 0.2],
                 'classifier__max_depth': [2, 3],
                 'classifier__min_samples_split': [2, 3],
                 'classifier__n_estimators': [100, 125, 150, 175]},
                {'classifier': [xgb.XGBClassifier(objective='multi:softmax', num_class=3)],
                 'classifier__learning_rate': [0.2, 0.3, 0.4],
                 'classifier__gamma': [0, 1],
                 'classifier__lambda': [1, 2, 3],
                 'classifier__n_estimators': [100, 125, 150, 175]},
                {'classifier': [SVC(gamma='auto')]}]
clf = GridSearchCV(pipe, search_space, verbose=5)
clf.fit(train_features, train_labels)
y_pred = clf.predict(dev)
print(clf.best_estimator_)
print(metrics.accuracy_score(dev_labels, y_pred))

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_features, train_labels)
y_pred = clf.predict(dev_features)
accuracy = accuracy_score(dev_labels, y_pred)
print("Accuracy:", accuracy)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

In [ ]:
#Under sampling
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'not minority')
train_features_resampled, train_labels_resampled = rus.fit_resample(train_features, train_labels)
rf = RandomForestClassifier()
rf.fit(train_features_resampled, train_labels_resampled)
y_pred = rf.predict(dev_features)
accuracy = accuracy_score(dev_labels, y_pred)
print("Accuracy:", accuracy)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

In [ ]:
#Over sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE()
train_features_resampled, train_labels_resampled = smote.fit_resample(train_features, train_labels)
rf = RandomForestClassifier()
rf.fit(train_features_resampled, train_labels_resampled)
y_pred = rf.predict(dev_features)
accuracy = accuracy_score(dev_labels, y_pred)
print("Accuracy:", accuracy)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

k_best_selector = SelectKBest(chi2, k=40)
train_features_new = k_best_selector.fit_transform(train_features, train_labels)
rf = RandomForestClassifier()
rf.fit(train_features_new, train_labels)
y_pred = rf.predict(dev_features[k_best_selector.get_feature_names_out()])
accuracy = accuracy_score(dev_labels, y_pred)
print("Accuracy:", accuracy)
confusion_matrix(y_true=dev_labels, y_pred=y_pred, labels=[0,1,2])

In [ ]:
def train_with_2_svc(features: pd.DataFrame, labels: pd.Series):
    category_not_2_features = features[labels != 2]
    category_not_2_labels = labels[labels != 2]
    